In [19]:
from sentence_transformers import SentenceTransformer


embedder_model = SentenceTransformer('BAAI/llm-embedder')

def embed_sentence(embedder_model: SentenceTransformer, sentence: str) -> list[float]:
    return embedder_model.encode(sentence, output_value="sentence_embedding", normalize_embeddings=True).tolist()

In [20]:
import re

def parse_faq_file(filepath: str) -> list[str]:
    with open(filepath, "r") as f:
        faq_data = f.read()

    pattern = re.compile(r'Q:(.*)\nA:(.*)')
    parsed_faq_data = re.findall(pattern , faq_data)

    refactored_faq_data = [f"question:{question.strip().lower()} answer:{answer.strip().lower()}" for question, answer in parsed_faq_data]
    return refactored_faq_data

faq_path = "data/Anadea homework -Tee Customizer FAQ.txt"
faq_data = parse_faq_file(faq_path)

In [41]:
import chromadb

# Initialize ChromaDB
chroma_client = chromadb.PersistentClient(path="../chroma_db")  # Persistent storage
chroma_client.delete_collection("faq_collection")
collection = chroma_client.get_or_create_collection(name="faq_collection")


# Store in ChromaDB
for i, qa in enumerate(faq_data):
    embedding = embed_sentence(embedder_model, qa)
    collection.add(
        ids=str(i),
        embeddings=embedding,
        documents=qa
    )

In [58]:
def retrieve_answer(embedder_model: SentenceTransformer, collection: chromadb.Collection, query: str, n_results=3) -> list[tuple[str, float]]:
    query_embedding = embedder_model.encode(query.strip().lower(), normalize_embeddings=True).tolist()
    closest_documents = collection.query(query_embeddings=query_embedding, n_results=n_results, include=["documents", "distances"])
    return [pair for pair in zip(closest_documents["documents"][0], closest_documents["distances"][0])]



In [59]:
query = "How can i change the color of the t-shirt?"

retrieved_answers = retrieve_answer(embedder_model, collection, query)

In [61]:
context = ""
for i in retrieved_answers:
    context += i[0] + "\n"

In [63]:
print(context)

question:can i add my own design to the t-shirt? answer:yes, you can upload your design or choose from our library of designs.
question:how do i care for my custom t-shirt? answer:we recommend washing your t-shirt inside out in cold water and tumble drying on low to preserve the print.
question:can i order matching t-shirts for an event? answer:yes, you can design and order matching t-shirts for events, teams, and groups.

